In [16]:
import pandas as pd

# Lecture du fichier
#df = pd.read_excel("crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012.xlsx", sheet_name=None)
# Charger le fichier Excel
file_path = 'crimes-et-delits-enregistres-par-les-services-de-gendarmerie-et-de-police-depuis-2012.xlsx'
workbook = pd.ExcelFile(file_path)

def safe_int(val):
    try:
        return int(val)
    except (ValueError, TypeError):
        return None



In [19]:
import re  # Import manquant
import pandas as pd
# Relancer l'extraction avec l'import ajouté
flattened_data = []

# Filtrer les feuilles contenant "PN"
sheet_names = [sheet for sheet in workbook.sheet_names if "PN" in sheet]

for sheet in sheet_names:
    match = re.match(r"Services PN (\d{4})", sheet)
    if not match:
        continue
    year = int(match.group(1))
    
    df = workbook.parse(sheet, header=None)

    # Extraire les lignes clés
    peripheriques = df.iloc[1, 2:]  # Ligne 2, colonnes C -> fin
    libelles_index = df.iloc[2, 2:]  # Ligne 3, colonnes C -> fin
    titres_crimes = df.iloc[4:, 1]  # Colonne B à partir de ligne 5
    donnees = df.iloc[4:, 2:]  # Données à partir de ligne 5, colonnes C -> fin
    departements = df.iloc[0, 2:]  # Ligne 1, colonnes C -> fin

    # Parcourir les colonnes de données
    for col_idx, (departement_header, peripherique) in enumerate(zip(departements, peripheriques)):
        libelle = libelles_index.iloc[col_idx]
        departement = str(departement_header).strip()

        for row_idx, valeur in enumerate(donnees.iloc[:, col_idx]):
            flattened_data.append({
                "Année": year,
                "Libellé index": libelle,
                "Titre crime": titres_crimes.iloc[row_idx],
                "Département": departement,
                "Service": "Police",
                "Périmètre": peripherique,
                "Valeur": safe_int(valeur)
            })

# Créer le DataFrame final
df_flattened_pn = pd.DataFrame(flattened_data)

# Afficher les 10 premières lignes
df_flattened_pn.head(10)


,Année,Libellé index,Titre crime,Département,Service,Périmètre,Valeur
0,2012,DIDPAF PREVESSIN 01,Homicides pour voler et à l'occasion de vols,01,Police,DCPAF,0
1,2012,DIDPAF PREVESSIN 01,Homicides pour d'autres motifs,01,Police,DCPAF,0
2,2012,DIDPAF PREVESSIN 01,Tentatives d'homicides pour voler et à l'occas...,01,Police,DCPAF,0
3,2012,DIDPAF PREVESSIN 01,Tentatives homicides pour d'autres motifs,01,Police,DCPAF,0
4,2012,DIDPAF PREVESSIN 01,Coups et blessures volontaires suivis de mort,01,Police,DCPAF,0
5,2012,DIDPAF PREVESSIN 01,Autres coups et blessures volontaires criminel...,01,Police,DCPAF,0
6,2012,DIDPAF PREVESSIN 01,Prises d'otages à l'occasion de vols,01,Police,DCPAF,0
7,2012,DIDPAF PREVESSIN 01,Prises d'otages dans un autre but,01,Police,DCPAF,0
8,2012,DIDPAF PREVESSIN 01,Sequestrations,01,Police,DCPAF,0
9,2012,DIDPAF PREVESSIN 01,Menaces ou chantages pour extorsion de fonds,01,Police,DCPAF,0


In [18]:
df_flattened_pn.shape

(729386, 7)

In [20]:
# Filtrer les feuilles contenant "GN"
gn_sheet_names = [sheet for sheet in workbook.sheet_names if "GN" in sheet]

flattened_data_gn = []

for sheet in gn_sheet_names:
    match = re.match(r"Services GN (\d{4})", sheet)
    if not match:
        continue
    year = int(match.group(1))

    df = workbook.parse(sheet, header=None)

    # Récupération des métadonnées
    departements = df.iloc[0, 2:]  # Ligne 1 -> Département
    libelles_index = df.iloc[1, 2:]  # Ligne 2 -> Libellé index \ CGD
    titres_crimes = df.iloc[4:, 1]  # Colonne B à partir de ligne 5
    donnees = df.iloc[4:, 2:]  # Données à partir de ligne 5, colonnes C -> fin

    for col_idx, departement_header in enumerate(departements):
        libelle = libelles_index.iloc[col_idx]
        departement = str(departement_header).strip()

        for row_idx, valeur in enumerate(donnees.iloc[:, col_idx]):
            flattened_data_gn.append({
                "Année": year,
                "Libellé index": libelle,
                "Titre crime": titres_crimes.iloc[row_idx],
                "Département": departement,
                "Service": "Gendarmerie",
                "Périmètre": None,
                "Valeur": safe_int(valeur)
            })

# Créer le DataFrame final GN
df_flattened_gn = pd.DataFrame(flattened_data_gn)

# Afficher les 10 premières lignes
df_flattened_gn.head(-1110)


,Année,Libellé index,Titre crime,Département,Service,Périmètre,Valeur
0,2012,CGD BELLEY,Homicides pour d'autres motifs,01,Gendarmerie,None,0.0
1,2012,CGD BELLEY,Tentatives d'homicides pour voler et à l'occas...,01,Gendarmerie,None,0.0
2,2012,CGD BELLEY,Tentatives homicides pour d'autres motifs,01,Gendarmerie,None,3.0
3,2012,CGD BELLEY,Coups et blessures volontaires suivis de mort,01,Gendarmerie,None,0.0
4,2012,CGD BELLEY,Autres coups et blessures volontaires criminel...,01,Gendarmerie,None,227.0
...,...,...,...,...,...,...,...
394105,2021,CGD ST BENOIT,Autres vols simples contre des particuliers da...,974,Gendarmerie,None,151.0
394106,2021,CGD ST BENOIT,Recels,974,Gendarmerie,None,21.0
394107,2021,CGD ST BENOIT,Proxénétisme,974,Gendarmerie,None,0.0
394108,2021,CGD ST BENOIT,Viols sur des majeur(e)s,974,Gendarmerie,None,29.0


In [21]:
# Concaténer les deux DataFrames PN et GN
df_final = pd.concat([df_flattened_pn, df_flattened_gn], ignore_index=True)

# Afficher les 10 premières lignes
df_final.head(10)

,Année,Libellé index,Titre crime,Département,Service,Périmètre,Valeur
0,2012,DIDPAF PREVESSIN 01,Homicides pour voler et à l'occasion de vols,01,Police,DCPAF,0.0
1,2012,DIDPAF PREVESSIN 01,Homicides pour d'autres motifs,01,Police,DCPAF,0.0
2,2012,DIDPAF PREVESSIN 01,Tentatives d'homicides pour voler et à l'occas...,01,Police,DCPAF,0.0
3,2012,DIDPAF PREVESSIN 01,Tentatives homicides pour d'autres motifs,01,Police,DCPAF,0.0
4,2012,DIDPAF PREVESSIN 01,Coups et blessures volontaires suivis de mort,01,Police,DCPAF,0.0
5,2012,DIDPAF PREVESSIN 01,Autres coups et blessures volontaires criminel...,01,Police,DCPAF,0.0
6,2012,DIDPAF PREVESSIN 01,Prises d'otages à l'occasion de vols,01,Police,DCPAF,0.0
7,2012,DIDPAF PREVESSIN 01,Prises d'otages dans un autre but,01,Police,DCPAF,0.0
8,2012,DIDPAF PREVESSIN 01,Sequestrations,01,Police,DCPAF,0.0
9,2012,DIDPAF PREVESSIN 01,Menaces ou chantages pour extorsion de fonds,01,Police,DCPAF,0.0


In [22]:
df_final.shape

(1124606, 7)

In [23]:
# Sauvegarder le DataFrame final en CSV
df_final.to_csv("crimes_aplatis.csv", index=False, encoding="utf-8-sig")